TPCH Query 6 and Query 5 runs

In [1]:
!pip install litendb

In [2]:
from platform import python_version
print(python_version())

3.12.3


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()
sc = spark.sparkContext

25/11/13 16:29:21 WARN Utils: Your hostname, flexdevbox resolves to a loopback address: 127.0.1.1; using 192.168.1.236 instead (on interface wlo1)
25/11/13 16:29:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/13 16:29:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc.range(1000).count()

1000

In [5]:
import os
from huggingface_hub import snapshot_download
#tpch_dataset = 'tpch-sfdot001g' # for sf=0.001g 
tpch_dataset = 'tpch-sf1g' # for sf=1g
local_dir=f'./{tpch_dataset}/'
if not os.path.exists(local_dir):
    print(f'Downloading dataset into {local_dir} ...')
    snapshot_download(
        repo_id="hkverma/"+tpch_dataset,
        repo_type="dataset",
        local_dir=local_dir,
        local_dir_use_symlinks=False  # ensures real copies, not symlinks
    )
    print('Download Complete.')
else:
    print(f"Dataset already exists in {local_dir}, skipping download.")  

Dataset already exists in ./tpch-sf1g/, skipping download.


In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, DoubleType

In [7]:
schema = StructType([StructField('L_ORDERKEY',IntegerType(),True),
                     StructField('L_PARTKEY',IntegerType(),True),
                     StructField('L_SUPPKEY',IntegerType(),True),
                     StructField('L_LINENUMBER',IntegerType(),True),
                     StructField('L_QUANTITY',IntegerType(),True),
                     StructField('L_EXTENDEDPRICE',DoubleType(),True),
                     StructField('L_DISCOUNT',DoubleType(),True),
                     StructField('L_TAX',DoubleType(),True),
                     StructField('L_RETURNFLAG',StringType(),True),
                     StructField('L_LINESTATUS',StringType(),True),
                     StructField('L_SHIPDATE',DateType(),True),
                     StructField('L_COMMITDATE',DateType(),True),
                     StructField('L_RECEIPTDATE',DateType(),True),
                     StructField('L_SHIPINSTRUCT',StringType(),True),
                     StructField('L_SHIPMODE',StringType(),True),
                     StructField('L_COMMENT',StringType(),True)])
schema

StructType([StructField('L_ORDERKEY', IntegerType(), True), StructField('L_PARTKEY', IntegerType(), True), StructField('L_SUPPKEY', IntegerType(), True), StructField('L_LINENUMBER', IntegerType(), True), StructField('L_QUANTITY', IntegerType(), True), StructField('L_EXTENDEDPRICE', DoubleType(), True), StructField('L_DISCOUNT', DoubleType(), True), StructField('L_TAX', DoubleType(), True), StructField('L_RETURNFLAG', StringType(), True), StructField('L_LINESTATUS', StringType(), True), StructField('L_SHIPDATE', DateType(), True), StructField('L_COMMITDATE', DateType(), True), StructField('L_RECEIPTDATE', DateType(), True), StructField('L_SHIPINSTRUCT', StringType(), True), StructField('L_SHIPMODE', StringType(), True), StructField('L_COMMENT', StringType(), True)])

In [8]:
lineitemDf = spark.read.format('csv').options(header='true').options(delimiter='|').schema(schema).load(f"{local_dir}/lineitem.csv")
lineitemDf.printSchema()
lineitemDf.show(5)
lineitemDf.createOrReplaceTempView("lineitem")

root
 |-- L_ORDERKEY: integer (nullable = true)
 |-- L_PARTKEY: integer (nullable = true)
 |-- L_SUPPKEY: integer (nullable = true)
 |-- L_LINENUMBER: integer (nullable = true)
 |-- L_QUANTITY: integer (nullable = true)
 |-- L_EXTENDEDPRICE: double (nullable = true)
 |-- L_DISCOUNT: double (nullable = true)
 |-- L_TAX: double (nullable = true)
 |-- L_RETURNFLAG: string (nullable = true)
 |-- L_LINESTATUS: string (nullable = true)
 |-- L_SHIPDATE: date (nullable = true)
 |-- L_COMMITDATE: date (nullable = true)
 |-- L_RECEIPTDATE: date (nullable = true)
 |-- L_SHIPINSTRUCT: string (nullable = true)
 |-- L_SHIPMODE: string (nullable = true)
 |-- L_COMMENT: string (nullable = true)

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+-----------------+----------+--------------------+
|L_ORDERKEY|L_PARTKEY|L_SUPPKEY|L_LINENUMBER|L_QUANTITY|L_EXTENDEDPRICE|L_DISCOUNT|L_TAX|L_RETURNFLAG|L_LIN

In [9]:
sqlDf = spark.sql("select sum(l_extendedprice * l_discount) as revenue from lineitem "
                  "where l_shipdate >= date '1997-01-01' "
                  "and l_shipdate < date '1997-01-01' + interval '1' year "
                  " and l_discount between 0.07 - 0.01 and 0.07 + 0.01 "
                  " and l_quantity < 25;")
sqlDf.show()

[Stage 2:>                                                        (0 + 20) / 20]

+--------------------+
|             revenue|
+--------------------+
|1.5659409560960007E8|
+--------------------+



Following neews to be done for the sql code shown below. load customer, orders, lineitem, supplier, nation, region tables. Then run query 5.

In [10]:
customerDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/customer.csv")
customerDf.printSchema()
customerDf.show(5)
customerDf.createOrReplaceTempView("customer")

root
 |-- C_CUSTKEY: integer (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: integer (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: double (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|C_CUSTKEY|            C_NAME|           C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|        1|Customer#000000001|   IVhzIApeRb ot,c,E|         15|25-989-741-2988|   711.56|    BUILDING|to the even, regu...|
|        2|Customer#000000002|XSTf4,NCwDVaWNe6t...|         13|23-768-687-3665|   121.65|  AUTOMOBILE|l accounts. blith...|
|        3|Customer#000000003|        MG9kdT

In [11]:
ordersDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/orders.csv")
ordersDf.printSchema()
ordersDf.show(5)
ordersDf.createOrReplaceTempView("orders")

root
 |-- O_ORDERKEY: integer (nullable = true)
 |-- O_CUSTKEY: integer (nullable = true)
 |-- O_ORDERSTATUS: string (nullable = true)
 |-- O_TOTALPRICE: double (nullable = true)
 |-- O_ORDERDATE: date (nullable = true)
 |-- O_ORDERPRIORITY: string (nullable = true)
 |-- O_CLERK: string (nullable = true)
 |-- O_SHIPPRIORITY: integer (nullable = true)
 |-- O_COMMENT: string (nullable = true)

+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|O_ORDERKEY|O_CUSTKEY|O_ORDERSTATUS|O_TOTALPRICE|O_ORDERDATE|O_ORDERPRIORITY|        O_CLERK|O_SHIPPRIORITY|           O_COMMENT|
+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|         1|    36901|            O|   173665.47| 1996-01-02|          5-LOW|Clerk#000000951|             0|nstructions sleep...|
|         2|    78002|            O|    46929.18| 1996-12-01|       1-URGENT|Clerk#00

In [12]:
supplierDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/supplier.csv")
supplierDf.printSchema()
supplierDf.show(5)
supplierDf.createOrReplaceTempView("supplier")

root
 |-- S_SUPPKEY: integer (nullable = true)
 |-- S_NAME: string (nullable = true)
 |-- S_ADDRESS: string (nullable = true)
 |-- S_NATIONKEY: integer (nullable = true)
 |-- S_PHONE: string (nullable = true)
 |-- S_ACCTBAL: double (nullable = true)
 |-- S_COMMENT: string (nullable = true)

+---------+------------------+--------------------+-----------+---------------+---------+--------------------+
|S_SUPPKEY|            S_NAME|           S_ADDRESS|S_NATIONKEY|        S_PHONE|S_ACCTBAL|           S_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+--------------------+
|        1|Supplier#000000001| N kD4on9OM Ipw3,...|         17|27-918-335-1736|  5755.94|each slyly above ...|
|        2|Supplier#000000002|89eJ5ksX3ImxJQBvx...|          5|15-679-861-2259|  4032.68| slyly bold instr...|
|        3|Supplier#000000003|q1,G3Pj6OjIuUYfUo...|          1|11-383-516-1199|   4192.4|blithely silent r...|
|        4|Supplier#000000004|Bk7ah4CK8SYQ

In [13]:
nationDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/nation.csv")
nationDf.printSchema()
nationDf.show(5)
nationDf.createOrReplaceTempView("nation")

root
 |-- N_NATIONKEY: integer (nullable = true)
 |-- N_NAME: string (nullable = true)
 |-- N_REGIONKEY: integer (nullable = true)
 |-- N_COMMENT: string (nullable = true)

+-----------+---------+-----------+--------------------+
|N_NATIONKEY|   N_NAME|N_REGIONKEY|           N_COMMENT|
+-----------+---------+-----------+--------------------+
|          0|  ALGERIA|          0| haggle. carefull...|
|          1|ARGENTINA|          1|al foxes promise ...|
|          2|   BRAZIL|          1|y alongside of th...|
|          3|   CANADA|          1|eas hang ironic, ...|
|          4|    EGYPT|          4|y above the caref...|
+-----------+---------+-----------+--------------------+
only showing top 5 rows



In [14]:
regionDf = spark.read.format('csv').options(header='true').options(delimiter='|').options(inferSchema='true').load(f"{local_dir}/region.csv")
regionDf.printSchema()
regionDf.show(5)
regionDf.createOrReplaceTempView("region")

root
 |-- R_REGIONKEY: integer (nullable = true)
 |-- R_NAME: string (nullable = true)
 |-- R_COMMENT: string (nullable = true)

+-----------+-----------+--------------------+
|R_REGIONKEY|     R_NAME|           R_COMMENT|
+-----------+-----------+--------------------+
|          0|     AFRICA|lar deposits. bli...|
|          1|    AMERICA|hs use ironic, ev...|
|          2|       ASIA|ges. thinly even ...|
|          3|     EUROPE|ly final courts c...|
|          4|MIDDLE EAST|uickly special ac...|
+-----------+-----------+--------------------+



In [15]:
sql5Df = spark.sql("select n_name, sum(l_extendedprice * (1 - l_discount)) as revenue "
    "from customer, orders, lineitem, supplier, nation, region "
    "where c_custkey = o_custkey "
    "and l_orderkey = o_orderkey "
    "and l_suppkey = s_suppkey "
    "and c_nationkey = s_nationkey "
    "and s_nationkey = n_nationkey "
    "and n_regionkey = r_regionkey "
    "and r_name = 'EUROPE' "
    "and o_orderdate >= date '1995-01-01' "
    "and o_orderdate < date '1995-01-01' + interval '1' year "
    "group by n_name order by revenue desc")
sql5Df.show()

+--------------+-------------------+
|        n_name|            revenue|
+--------------+-------------------+
|        RUSSIA|     5.6063022839E7|
|       ROMANIA|    5.49946485948E7|
|UNITED KINGDOM|     5.4686147489E7|
|        FRANCE|5.194113723340001E7|
|       GERMANY|    5.15364981838E7|
+--------------+-------------------+

